In [1]:
%cd ..

C:\Users\hp\dev\video-membership-app


In [2]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table

In [3]:
db.get_session()
sync_table(User)

In [4]:
post_data = {
    "email": "user@gmail.com",
    "password": "123456"
}

In [12]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [10]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [11]:
UserLoginSchema(**post_data)

UserLoginSchema(email='user@gmail.com', password=SecretStr('**********'))

In [23]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr
    
    @validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        if q.count() != 0:
            raise ValueError("Email is not available")
        return v
        
    @validator("password_confirm")
    def password_match(cls, v, values, **kwargs):
        password = values.get("password")
        password_confirm = v
        if password != password_confirm:
            raise ValueError("Password do not match")
        
        return v

In [24]:
UserSignupSchema(email="client@gmail.com", password="123456", password_confirm="1234")

ValidationError: 2 validation errors for UserSignupSchema
email
  Email is not available (type=value_error)
password_confirm
  Password do not match (type=value_error)

In [15]:
UserSignupSchema(email="user1@gmail.com", password="123456", password_confirm="1234")

ValidationError: 1 validation error for UserSignupSchema
password_confirm
  Password do not match (type=value_error)

In [18]:
data = UserSignupSchema(email="user1@gmail.com", password="123456", password_confirm="123456")

In [19]:
data.dict()

{'email': 'user1@gmail.com',
 'password': SecretStr('**********'),
 'password_confirm': SecretStr('**********')}

In [22]:
data.dict()["password_confirm"].get_secret_value()

'123456'

In [20]:
data.json()

'{"email": "user1@gmail.com", "password": "**********", "password_confirm": "**********"}'

In [25]:
dir(data.dict()["password"])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__modify_schema__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_secret_value',
 'display',
 'get_secret_value',
 'max_length',
 'min_length',
 'validate']